## Xử lý dữ liệu mạng xã hội Twitter
- Liệt kê 20 từ được nhắc nhiều nhất
- Liệt kê 10 tài khoản được nhắc nhiều nhất
- Tìm khoảng thời gian trong ngày (0-24 giờ) tác giả đăng bài nhiều nhất

In [1]:
# Khai báo thư viện pandas
import pandas as pd

In [2]:
# Mở file .csv & nạp dữ liệu vào dataframe:
df = pd.read_csv('Downloads/elonmusk_tweets.csv')

In [3]:
df.head()

,id,created_at,text
0,849636868052275200,2017-04-05 14:56:29,b'And so the robots spared humanity ... https:...
1,848988730585096192,2017-04-03 20:01:01,"b""@ForIn2020 @waltmossberg @mims @defcon_5 Exa..."
2,848943072423497728,2017-04-03 16:59:35,"b'@waltmossberg @mims @defcon_5 Et tu, Walt?'"
3,848935705057280001,2017-04-03 16:30:19,b'Stormy weather in Shortville ...'
4,848416049573658624,2017-04-02 06:05:23,"b""@DaveLeeBBC @verge Coal is dying due to nat ..."


In [4]:
# Lấy tất cả dòng tweets & nối thành 1 chuỗi
tweets = " ".join([s for s in df['text']])

In [5]:
# Tạo cấu trúc dạng từ điển chứa số lượng của mỗi từ trong tập tweets

word_counts = {}

# Xét từng từ một trong chuỗi tweets:
for w in tweets.split():
    # nếu từ w chưa có trong từ điển thì bổ sung w vào từ điển & khởi tạo số lượng = 1
    if w not in word_counts: 
        word_counts[w] = 1
    # ngược lại, nếu từ w đã có trong từ điển thì tăng số lượng thêm 1
    else:
        word_counts[w] += 1

In [6]:
# Để lấy top từ xuất hiện nhiều nhất, một cách giải quyết là sắp xếp từ điển theo thứ tự giảm dần của <giá trị>
# sau đó lấy ra n từ đầu tiên

sorted_word_counts = sorted(word_counts.items(), key = lambda x : x[1], reverse=True)

In [7]:
n_top_words = 50

top_words = [w for w,c in sorted_word_counts[:n_top_words]]

print("Top %d từ xuất hiện nhiều nhất:" % n_top_words)
print(top_words)

Top 50 từ xuất hiện nhiều nhất:
['the', 'to', 'of', 'a', 'in', 'is', 'for', 'on', 'and', "b'RT", 'be', 'at', 'Tesla', 'that', 'will', 'Model', 'I', 'by', 'but', 'it', '&amp;', 'with', 'are', 'from', 'this', 'S', 'we', 'was', 'as', 'not', 'like', 'about', 'you', '@SpaceX:', 'an', 'launch', 'rocket', 'just', 'all', 'b"RT', 'have', 'Falcon', 'my', '@TeslaMotors:', 'so', 'has', 'or', 'more', 'car', 'do']


In [8]:
# Liệt kê n tài khoản được nhắc đến nhiều nhất

# Tạo cấu trúc dạng từ điển chứa số lần xuất hiện của mỗi tài khoản trong các tweets

account_counts = {}

# Xét từng từ một trong chuỗi tweets:
for w in tweets.split():
    if "@" in w:
        if w not in account_counts: 
            account_counts[w] = 1
        else:
            account_counts[w] += 1

In [9]:
# Sắp xếp tập từ điển theo thứ tự giảm dần của giá trị

sorted_account_counts = sorted(account_counts.items(), key = lambda x : x[1], reverse=True)

In [10]:
n_top_account = 20

top_accounts = [w for w,c in sorted_account_counts[:n_top_account]]

print("Top %d tài khoản xuất hiện nhiều nhất:" % n_top_account)
print(top_accounts)

Top 20 tài khoản xuất hiện nhiều nhất:
['@SpaceX:', '@TeslaMotors:', '@TeslaMotors', '@elonmusk', '@SpaceX', '@NASA', '@NASA:', "b'@vicentes", "b'@ID_AA_Carmack", "b'@elonmusk", '@WIRED:', '@FredericLambert', '@Space_Station', '@FortuneMagazine', '@teslamotors', '@', '@OpenAI:', "b'@MacTechGenius", '@TheOnion:', "b'@FredericLambert"]


In [11]:
# Tìm khoảng thời gian trong ngày (0-24 giờ) tác giả đăng bài nhiều nhất.

df.head()

,id,created_at,text
0,849636868052275200,2017-04-05 14:56:29,b'And so the robots spared humanity ... https:...
1,848988730585096192,2017-04-03 20:01:01,"b""@ForIn2020 @waltmossberg @mims @defcon_5 Exa..."
2,848943072423497728,2017-04-03 16:59:35,"b'@waltmossberg @mims @defcon_5 Et tu, Walt?'"
3,848935705057280001,2017-04-03 16:30:19,b'Stormy weather in Shortville ...'
4,848416049573658624,2017-04-02 06:05:23,"b""@DaveLeeBBC @verge Coal is dying due to nat ..."


In [12]:
# Một cách giải quyết "chân phương": 

def getHour(s):
    return s[11:13]

In [13]:
# Tạo thêm 1 cột mới của dataframe đặt tên là 'created_hour' chứa giờ đăng bài

df['created_hour'] = df['created_at'].apply(getHour)

In [14]:
df.head()

,id,created_at,text,created_hour
0,849636868052275200,2017-04-05 14:56:29,b'And so the robots spared humanity ... https:...,14
1,848988730585096192,2017-04-03 20:01:01,"b""@ForIn2020 @waltmossberg @mims @defcon_5 Exa...",20
2,848943072423497728,2017-04-03 16:59:35,"b'@waltmossberg @mims @defcon_5 Et tu, Walt?'",16
3,848935705057280001,2017-04-03 16:30:19,b'Stormy weather in Shortville ...',16
4,848416049573658624,2017-04-02 06:05:23,"b""@DaveLeeBBC @verge Coal is dying due to nat ...",06


In [15]:
# Tạo từ điển đếm số lượng bài tweets theo khung giờ

post_hours = df['created_hour']

In [16]:
post_hour_counts = {}

for h in post_hours:
    if h not in post_hour_counts: 
        post_hour_counts[h] = 1
    # ngược lại, nếu từ w đã có trong từ điển thì tăng số lượng thêm 1
    else:
        post_hour_counts[h] += 1

In [17]:
sorted_post_hours = sorted(post_hour_counts.items(), key=lambda x:x[1], reverse=True)

In [18]:
sorted_post_hours

[('16', 198),
 ('21', 192),
 ('17', 190),
 ('15', 190),
 ('01', 183),
 ('20', 176),
 ('22', 168),
 ('18', 162),
 ('19', 154),
 ('00', 144),
 ('23', 141),
 ('02', 119),
 ('14', 114),
 ('03', 104),
 ('04', 98),
 ('07', 94),
 ('06', 85),
 ('08', 82),
 ('05', 81),
 ('09', 55),
 ('10', 34),
 ('13', 23),
 ('12', 17),
 ('11', 15)]